In [1]:
import requests
import random
import os
import pandas as pd
from geopy.geocoders import Nominatim

In [2]:
# Google Street View Image API endpoint
API_ENDPOINT = "https://maps.googleapis.com/maps/api/streetview"
metadata_API_ENDPOINT = "https://maps.googleapis.com/maps/api/streetview/metadata"

# Your Google API key
API_KEY = #ADD YOUR API KEY

In [10]:
# Number of images per city
num_images_per_city = 100

# Directory to save the images
output_directory = "street_images"
output_directory_better = "street_images_plid"

In [6]:
# List of cities
cities = ["Bangkok", "Barcelona", "Boston", "Brussels", "Buenos Aires",
          "Chicago","Lisbon", "London", "Los Angeles", "Madrid", "Medellin", "Melbourne", "Mexico City", "Miami",
         "Minneapolis", "Osaka", "Oslo", "Phoenix", "Prague", "Paris", "Rome", "Toronto", "Washington"] 

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
if not os.path.exists(output_directory_better):
    os.makedirs(output_directory_better)

"""for city in cities:
    if not os.path.exists(output_directory+'/'+city):
        os.makedirs(output_directory+'/'+city)

for city in cities:
    if not os.path.exists(output_directory_better+'/'+city):
        os.makedirs(output_directory_better+'/'+city)"""

rev_mapping = {'BuenosAires': 'Buenos Aires', 'LosAngeles': 'Los Angeles', 'MexicoCity': 'Mexico City', 'OSL':'Oslo', 'PRG': 'Prague',
                'PRS': 'Paris', 'TRT':'Toronto', 'WashingtonDC':'Washington'}

for city in os.listdir('street_images_plid'):
    if city in rev_mapping:
        os.rename('street_images_plid/'+city, 'street_images_plid/'+rev_mapping[city])

In [7]:
def get_city_bb(city):
    geolocator = Nominatim(user_agent="random_street_images")
    try: 
        location = geolocator.geocode(city)
        if location:
            """lat = location.latitude
            lon = location.longitude
            # Define a bounding box around the city
            lat_min = float(lat) - 0.05  # Adjust the range as needed
            lat_max = float(lat) + 0.05
            lon_min = lon - 0.05
            lon_max = lon + 0.05 """

            bb= location.raw['boundingbox']
            # Define a bounding box around the city
            lat_min = float(bb[0]) 
            lat_max = float(bb[1])
            lon_min = float(bb[2])
            lon_max = float(bb[3])

            return [lat_min, lat_max, lon_min, lon_max]
        else:
            return None
    except:
        print("geolocator unavailable, using requests")
        location = requests.get(f"https://nominatim.openstreetmap.org/search?q={city}&format=json&limit=1")
        if (location.status_code) == 200:
            bb = location.json()[0]['boundingbox']
            lat_min = float(bb[0]) 
            lat_max = float(bb[1])
            lon_min = float(bb[2])
            lon_max = float(bb[3])
            return [lat_min, lat_max, lon_min, lon_max]
        else:
            return None

def random_location(bb):
    random_lat = round(random.uniform(bb[0], bb[1]), 4)
    random_lon = round(random.uniform(bb[2], bb[3]), 4)
    return random_lat, random_lon
    
    """geolocator = Nominatim(user_agent="random_street_images")
    try: 
        location = geolocator.geocode(city)
        if location:
            lat = location.latitude
            lon = location.longitude
            # Define a bounding box around the city
            lat_min = float(lat) - 0.05  # Adjust the range as needed
            lat_max = float(lat) + 0.05
            lon_min = lon - 0.05
            lon_max = lon + 0.05 

            bb= location.raw['boundingbox']
            # Define a bounding box around the city
            lat_min = float(bb[0]) 
            lat_max = float(bb[1])
            lon_min = float(bb[2])
            lon_max = float(bb[3])
            
            # Generate random latitude and longitude within the bounding box
            random_lat = random.uniform(lat_min, lat_max)
            random_lon = random.uniform(lon_min, lon_max)
            return random_lat, random_lon
        else:
            return None
    except:
        print("geolocator unavailable, using requests")
        location = requests.get(f"https://nominatim.openstreetmap.org/search?q={city}&format=json&limit=1")
        if (location.status_code) == 200:
            bb = location.json()[0]['boundingbox']
            lat_min = float(bb[0]) 
            lat_max = float(bb[1])
            lon_min = float(bb[2])
            lon_max = float(bb[3])
            random_lat = random.uniform(lat_min, lat_max)
            random_lon = random.uniform(lon_min, lon_max)
            return random_lat, random_lon
        else:
            return None"""
    
def check_metadata(params):
    response = requests.get(metadata_API_ENDPOINT, params=params)
    if response.json()['status'] == 'OK':
        return True, response.json()
    else:
        return False, response.json()
    
def random_sv_image(city, city_bb, i):
    """use if you want absolutely randomly dispersed data throughout a given city"""
    lat, lon = random_location(city_bb)
    
    params = {
            "location": (str(lat) + ',' + str(lon)),
            "size": "640x480",
            "fov": "90",
            "pitch": "0",
            #"heading": str(random.randint(0, 360)),
            "key": API_KEY,
            "return_error_code": "true"
        }
    
    meta, data = check_metadata(params)
    
    if meta:
    
        response = requests.get(API_ENDPOINT, params=params)

        if response.status_code == 200:
            image_filename = f"{city}/{city}_{data['pano_id']}_{str(data['location']['lat'])}_{str(data['location']['lng'])}.jpg"
            with open(os.path.join(output_directory, image_filename), "wb") as f:
                f.write(response.content)
                print(f"Saved image for {city} ({i+1}/{num_images_per_city})")
            return True
        else:
            print(f"Failed to fetch image for {city} ({i+1}/{num_images_per_city})")
            return False
    
    else:
        print(f"no street view image found for {city}, try again")
        return False

def quasi_random_sv_image(city, lat, lon, i):
    """use if you have trained on a specific set of place_ids and want somewhat random data near the training set"""    
    params = {
            "location": (str(lat) + ',' + str(lon)),
            "size": "640x480",
            "fov": "90",
            "pitch": "0",
            #"heading": str(random.randint(0, 360)),
            "key": API_KEY,
            "return_error_code": "true"
        }
    
    meta, data = check_metadata(params)
    
    if meta:
    
        response = requests.get(API_ENDPOINT, params=params)

        if response.status_code == 200:
            image_filename = f"{city}/{city}_{data['pano_id']}_{str(data['location']['lat'])}_{str(data['location']['lng'])}.jpg"
            with open(os.path.join(output_directory_better, image_filename), "wb") as f:
                f.write(response.content)
                print(f"Saved image for {city} ({i+1}/{num_images_per_city})")
            return True
        else:
            print(f"Failed to fetch image for {city} ({i+1}/{num_images_per_city})")
            return False
    
    else:
        print(f"no street view image found for {city}, try again")
        return False

In [8]:
def create_random_test_data(cities, n_per_city, err_tolerance):
    for city in cities:
        i = 0
        err = 0
        city_bb = get_city_bb(city)
        while i < n_per_city:
            res = random_sv_image(city, city_bb, i)
            if res == True:
                i += 1
            else:
                err += 1
                if err > err_tolerance:
                    print("too many failed attempts, exiting city: ", city)
                    break
        print('done with: ', city)

In [16]:
#create_random_test_data(cities, num_images_per_city, 150)

geolocator unavailable, using requests
Saved image for Bangkok (1/50)
no street view image found for Bangkok, try again
Saved image for Bangkok (2/50)
no street view image found for Bangkok, try again
no street view image found for Bangkok, try again
Saved image for Bangkok (3/50)
no street view image found for Bangkok, try again
no street view image found for Bangkok, try again
no street view image found for Bangkok, try again
no street view image found for Bangkok, try again
no street view image found for Bangkok, try again
Saved image for Bangkok (4/50)
no street view image found for Bangkok, try again
no street view image found for Bangkok, try again
no street view image found for Bangkok, try again
no street view image found for Bangkok, try again
no street view image found for Bangkok, try again
no street view image found for Bangkok, try again
Saved image for Bangkok (5/50)
no street view image found for Bangkok, try again
Saved image for Bangkok (6/50)
no street view image foun

## Using the Place Ids to build better test dataset

In [9]:
df = pd.DataFrame()
path_to_data = "../GSVData/Dataframes"
for csv in os.listdir(path_to_data):
    if df.empty:
        df = pd.read_csv(os.path.join(path_to_data,csv))
    else:
        temp = pd.read_csv(os.path.join(path_to_data, csv))
        df = pd.concat([df, temp])

In [11]:
def create_quasi_random_test_data(cities, n_per_city, err_tolerance, df):
    mappings = {'Buenos Aires': 'BuenosAires', 'Los Angeles': 'LosAngeles', 'Mexico City': 'MexicoCity', 'Oslo': 'OSL', 'Prague': 'PRG',
                'Paris': 'PRS', 'Toronto':'TRT', 'Washington':'WashingtonDC'}
    for city in cities:
        i = 0
        err = 0
        if city in mappings:
            city_df = df[df['city_id'] == mappings[city]].drop_duplicates(subset='place_id')
        else: 
            city_df = df[df['city_id'] == city].drop_duplicates(subset='place_id')
        while i < n_per_city:
            rand_plid = city_df.sample(1)
            lat, lon = round(rand_plid.iloc[0]['lat'], 5), round(rand_plid.iloc[0]['lon'], 5)
            res = quasi_random_sv_image(city, lat, lon, i)
            if res == True:
                i += 1
            else:
                err += 1
                if err > err_tolerance:
                    print("too many failed attempts, exiting city: ", city)
                    break
        print('done with: ', city)

In [12]:
create_quasi_random_test_data(cities, num_images_per_city, 100, df)

Saved image for Bangkok (1/100)
Saved image for Bangkok (2/100)
Saved image for Bangkok (3/100)
Saved image for Bangkok (4/100)
Saved image for Bangkok (5/100)
Saved image for Bangkok (6/100)
Saved image for Bangkok (7/100)
Saved image for Bangkok (8/100)
Saved image for Bangkok (9/100)
Saved image for Bangkok (10/100)
Saved image for Bangkok (11/100)
Saved image for Bangkok (12/100)
Saved image for Bangkok (13/100)
Saved image for Bangkok (14/100)
Saved image for Bangkok (15/100)
Saved image for Bangkok (16/100)
Saved image for Bangkok (17/100)
Saved image for Bangkok (18/100)
Saved image for Bangkok (19/100)
Saved image for Bangkok (20/100)
Saved image for Bangkok (21/100)
Saved image for Bangkok (22/100)
Saved image for Bangkok (23/100)
Saved image for Bangkok (24/100)
Saved image for Bangkok (25/100)
Saved image for Bangkok (26/100)
Saved image for Bangkok (27/100)
Saved image for Bangkok (28/100)
Saved image for Bangkok (29/100)
Saved image for Bangkok (30/100)
Saved image for Ban

In [10]:
df[df['city_id'] == 'PRG'].drop_duplicates(subset='place_id')

,place_id,year,month,northdeg,city_id,lat,lon,panoid
0,569,2019,3,285,PRG,50.052260,14.334993,oKGbeVrHvr3BMIXKUqVe8Q
1,231,2014,6,532,PRG,50.042684,14.433778,eQs-eZy63XBtDjxIRDqLEQ
2,1973,2017,6,408,PRG,50.081085,14.419345,ewqhgCclSitFVYQeifYsJg
3,1800,2011,8,381,PRG,50.078183,14.348998,gbcU6ZaULO4fI6t9wPEXZQ
4,2154,2012,4,456,PRG,50.084246,14.508941,lVkFWU4qLx3Fdg588orF5Q
...,...,...,...,...,...,...,...,...
16104,81,2019,9,488,PRG,50.039459,14.352086,ITeHR-M5avLCRI0x2dZytQ
16148,1196,2019,9,283,PRG,50.066669,14.361842,YKJbX-CXnv3WpgmuJ5UMbw
16431,1865,2014,5,421,PRG,50.077877,14.503948,sdXU0YXKJh6C5IO0J545kQ
16799,3053,2014,6,374,PRG,50.103442,14.511155,kYjz-f7tKtObqaVk8qgWMg


In [48]:
df[df['city_id'] == "Bangkok"].drop_duplicates(subset='place_id').sample(1).iloc[0]['lat']

13.76731137447785

In [38]:
params = {
            "location": (str(13.76226) + ',' + str(100.4937)),
            "size": "640x480",
            "fov": "90",
            "pitch": "0",
            #"heading": str(random.randint(0, 360)),
            "key": API_KEY,
            "return_error_code": "true"
        }
check_metadata(params)

(True,
 {'copyright': '© Google',
  'date': '2024-01',
  'location': {'lat': 13.76225659896977, 'lng': 100.4937308917267},
  'pano_id': 'y14TmHOgJdwPs1FmwGEusA',
  'status': 'OK'})

In [9]:
response = requests.get("https://nominatim.openstreetmap.org/search?q=Lisbon&format=json&limit=1")

In [15]:
response.json()[0]

{'place_id': 287756547,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 5400890,
 'lat': '38.7077507',
 'lon': '-9.1365919',
 'class': 'boundary',
 'type': 'administrative',
 'place_rank': 14,
 'importance': 0.7149698324141975,
 'addresstype': 'city',
 'name': 'Lisboa',
 'display_name': 'Lisboa, Portugal',
 'boundingbox': ['38.6913994', '38.7967584', '-9.2298356', '-9.0863328']}

In [13]:
city

'Washington'